# 使用geatpy优化RNN定位模型参数

In [1]:
import math
import numpy as np
import pandas as pd
import geatpy as ea
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, SimpleRNN, LSTM
from multiprocessing.dummy import Pool as ThreadPool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, explained_variance_score, r2_score, mean_squared_error

## 读取数据

In [2]:
# 加载训练数据
def load_data(path):
    df = pd.read_csv(path)
    dv = df.values
    X = dv[:, 1:-2]
    Y = dv[:, -2:]
    return X[:, :, np.newaxis], Y

In [3]:
X, y = load_data('./dataset/location_data.csv')
print(X.shape)

(2000, 100, 1)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2021)  # 训练测试划分

## 定义RNN模型

In [5]:
def RFID_RNN():
    model = Sequential()
    model.add(SimpleRNN(units=100,
                        # kernel_initializer=initializers.RandomNormal(stddev=0.001),
                        # recurrent_initializer=initializers.Identity(gain=1.0),
                        batch_input_shape=(None, 100, 1)))
    model.add(Dense(2))
    model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['mae'])

    # model.summary()
    return model

## 训练模型查看测试效果

In [6]:
%%time
model = RFID_RNN()
model.fit(X_train, y_train, epochs=10, batch_size=100)
loss = model.evaluate(X_test, y_test, batch_size=10) # 输出损失和MAE
print('损失与精度：', loss)
model.save('./models/LSTM_2021.h5')

pxy = model.predict(X_test)
print(pxy[:10])
print(y_test[:10])
MAE = mean_absolute_error(y_test, pxy)
MSE = mean_squared_error(y_test, pxy)
r2 = r2_score(y_test, pxy)
ev = explained_variance_score(y_test, pxy)
print('MAE指标:', MAE)
print('MSE指标:', MSE)
print('RMSE指标:', math.sqrt(MSE))
print('r2指标:', r2)
print('ev指标:', ev)


2021-12-21 15:15:18.798536: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-21 15:15:19.040347: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/10
16/16 [==============================] - 1s 35ms/step - loss: 19.4828 - mae: 3.6433
Epoch 2/10
16/16 [==============================] - 1s 33ms/step - loss: 7.3933 - mae: 2.3381
Epoch 3/10
16/16 [==============================] - 1s 32ms/step - loss: 6.0925 - mae: 2.0834
Epoch 4/10
16/16 [==============================] - 1s 32ms/step - loss: 5.2798 - mae: 1.9079
Epoch 5/10
16/16 [==============================] - 1s 32ms/step - loss: 4.6102 - mae: 1.7727
Epoch 6/10
16/16 [==============================] - 1s 32ms/step - loss: 4.6060 - mae: 1.7721
Epoch 7/10
16/16 [==============================] - 1s 32ms/step - loss: 4.4246 - mae: 1.7293
Epoch 8/10
16/16 [==============================] - 1s 32ms/step - loss: 4.2319 - mae: 1.6822
Epoch 9/10
16/16 [==============================] - 1s 34ms/step - loss: 4.2957 - mae: 1.6833
Epoch 10/10
40/40 [==============================] - 0s 5ms/step - loss: 4.3358 - mae: 1.7232
损失与精度： [4.335757732391357, 1.723159909248352]
[[4.7814503 5

## 使用进化算法调参

 ### 自定义问题类

In [7]:
class RFIDRNN(ea.Problem):
    def __init__(self, X_train, X_test, y_train, y_test):
        name = 'RFIDRNN'
        M = 1 # 初始化M（目标维数）
        maxormins = [-1] # 初始化maxormins（目标最小最大化标记列表，1：最小化该目标；-1：最大化该目标）
        Dim = 2 # 初始化Dim（决策变量维数）
        varTypes = np.array([0] * Dim) # 初始化varTypes 0-连续
        lb = [10, 50] # 决策变量下界
        ub = [500, 800] # 决策变量上界
        lbin = [1] * Dim # 决策变量下边界（0表示不包含该变量的下边界，1表示包含）
        ubin = [1] * Dim # 决策变量上边界（0表示不包含该变量的上边界，1表示包含）
        # 调用父类构造方法完成实例化
        ea.Problem.__init__(self, name, M, maxormins, Dim, varTypes, lb, ub, lbin, ubin)
        # 数据设置
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        
        
    # 目标函数，采用多线程加速计算
    def aimFunc(self, pop):
        Vars = pop.Phen # 得到决策变量矩阵
        # print(Vars)
        pop.ObjV = np.zeros((pop.sizes, 1)) # 初始化种群个体目标函数值列向量
        def subAimFunc(i):
            epochs = int(Vars[i, 0])
            batch_size = int(Vars[i, 1])
            model = RFID_RNN()
            model.fit(self.X_train, self.y_train, epochs=epochs, batch_size=batch_size, verbose=0)
            pxy = model.predict(self.X_test)
            pop.ObjV[i] = r2_score(self.y_test, pxy) # 最小化r2作为目标函数
        pool = ThreadPool(processes=2) # 设置池的大小
        pool.map(subAimFunc, list(range(pop.sizes))) # 散列种群每个个体进行加速计算
        
        
    # 代入优化后的参数对测试集进行检验，计算指标
    def test(self, epochs, batch_size):
        X02, y02 = load_data('./dataset/location_data02.csv')
        model = RFID_RNN()
        model.fit(self.X_train, self.y_train, epochs=epochs, batch_size=batch_size, verbose=0)
        model.save('./models/RNN_2021.h5')
        pxy = model.predict(X02)
        MAE = mean_absolute_error(y02, pxy)
        MSE = mean_squared_error(y02, pxy)
        r2 = r2_score(y02, pxy)
        ev = explained_variance_score(y02, pxy)
        print('MAE指标:', MAE)
        print('MSE指标:', MSE)
        print('RMSE指标', math.sqrt(MSE))
        print('r2指标:', r2)
        print('ev指标:', ev)
        

### 编写执行代码

In [ ]:
"""===============================实例化问题对象==========================="""

problem = RFIDRNN(X_train, X_test, y_train, y_test) # 生成问题对象

"""=================================种群设置==============================="""

Encoding = 'RI'       # 编码方式
NIND = 2             # 种群规模
Field = ea.crtfld(Encoding, problem.varTypes, problem.ranges, problem.borders) # 创建区域描述器
population = ea.Population(Encoding, Field, NIND) # 实例化种群对象（此时种群还没被初始化，仅仅是完成种群对象的实例化）

"""===============================算法参数设置============================="""

myAlgorithm = ea.soea_DE_rand_1_bin_templet(problem, population) # 实例化一个算法模板对象
myAlgorithm.MAXGEN = 10 # 最大进化代数
myAlgorithm.trappedValue = 1e-6 # “进化停滞”判断阈值
myAlgorithm.maxTrappedCount = 10 # 进化停滞计数器最大上限值，如果连续maxTrappedCount代被判定进化陷入停滞，则终止进化
myAlgorithm.logTras = 1  # 设置每隔多少代记录日志，若设置成0则表示不记录日志
myAlgorithm.verbose = True  # 设置是否打印输出日志信息
myAlgorithm.drawing = 1  # 设置绘图方式（0：不绘图；1：绘制结果图；2：绘制目标空间过程动画；3：绘制决策空间过程动画）

"""===========================调用算法模板进行种群进化======================="""

[BestIndi, population] = myAlgorithm.run()  # 执行算法模板，得到最优个体以及最后一代种群
BestIndi.save()  # 把最优个体的信息保存到文件中

"""==================================输出结果============================="""

print('用时：%f 秒' % myAlgorithm.passTime)
print('评价次数：%d 次' % myAlgorithm.evalsNum)
if BestIndi.sizes != 0:
    print('最优的目标函数值为：%s' % BestIndi.ObjV[0][0])
    print('最优的控制变量值为：')
    for i in range(BestIndi.Phen.shape[1]):
        print(BestIndi.Phen[0, i])
else:
    print('没找到可行解。')
    
"""=================================检验结果==============================="""

problem.test(epochs= int(BestIndi.Phen[0][0]), batch_size= int(BestIndi.Phen[0][1]))
